In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf # pip install tensorflow-macos
import os
tf.random.set_seed(1)

# The typical architecture of a Recurrent Neural Network (RNN)
- The premise of an RNN is simple: use information from the past to help you with the future (this is where the term recurrent comes from). In other words, take an input (X) and compute an output (y) based on all previous inputs.

When an RNN looks at a sequence of text (already in numerical form), the patterns it learns are continually updated based on the order of the sequence.

For a simple example, take two sentences:

1. Massive earthquake last week, no?
2. No massive earthquake last week.

Both contain exactly the same words but have different meaning. The order of the words determines the meaning (one could argue punctuation marks also dictate the meaning but for simplicity sake, let's stay focused on the words).

### actual architecture
1. input layer
2. text vectorization layer
3. embedding
4. RNN cell(s)
5. Hidden activation
6. pooling layer (sometimes, usually for Conv1D models)
7. fully connected layer
8. output layer



# Preparing a notebook for our first NLP with TensorFlow project
[data from kaggle](https://www.kaggle.com/competitions/nlp-getting-started/leaderboard)

In [2]:
from helper_functions import create_tensorboard_callback, unzip_data, plot_loss_curves, compare_historys

In [3]:
zip_path = "nlp_getting_started.zip"
if not os.path.isfile(zip_path):
    os.chdir("data")
unzip_data(zip_path)

# Becoming one with the data and visualizing a text dataset

In [4]:
# we can use pandas because the data isn't too big
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# shuffle training data
train_df = train_df.sample(frac=1, random_state=1)

In [5]:
train_df.head()

,id,keyword,location,text,target
3228,4632,emergency%20services,"Sydney, New South Wales",Goulburn man Henry Van Bilsen missing: Emergen...,1
3706,5271,fear,NaN,The things we fear most in organizations--fluc...,0
6957,9982,tsunami,Land Of The Kings,@tsunami_esh ?? hey Esh,0
2887,4149,drown,NaN,@POTUS you until you drown by water entering t...,0
7464,10680,wounds,"cody, austin follows ?*?",Crawling in my skin\nThese wounds they will no...,1


In [6]:
train_df.iloc[0]["text"]

'Goulburn man Henry Van Bilsen missing: Emergency services are searching for a Goulburn man who disappeared from his\x89Û_ http://t.co/z99pKJzTRp'

In [7]:
# test data frame looks the same but without targets
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
train_df["target"].value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [9]:
print(len(train_df), len(test_df))

7613 3263


In [10]:
# let's visualize some random samples
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df[["text", "target"]][random_index:random_index+5].itertuples():
    _, text, target = row
    print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
    print(f"Text:\n{text}\n")
    print("---\n")

Target: 0 (not real disaster)
Text:
NEWS FLASH!  Any decent billers been promoted to 'manager'? If so let me know as I want to watch your billings fall off a cliff.
#Humble

---

Target: 0 (not real disaster)
Text:
Some older Native Australians believe that the oceans were created from the urine of an angry god who tried to drown the world.

---

Target: 0 (not real disaster)
Text:
'Up to 40% of businesses affected by a natural or man-made disaster never reopen'
http://t.co/35JyAp0ul9

---

Target: 1 (real disaster)
Text:
The Catastrophic Effects of Hiroshima and Nagasaki Atomic Bombings Still Being Felt Today http://t.co/TzxeG4gOkD

---

Target: 0 (not real disaster)
Text:
@Uptown_Jorge head up like yo nose bleeding

---



# Splitting data into training and validation sets

In [11]:
from sklearn.model_selection import train_test_split
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df["text"].to_numpy(),
                                                                            train_df["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=1)

In [12]:
print(len(train_sentences), len(train_labels), len(val_sentences), len(val_labels))

6851 6851 762 762


In [13]:
train_sentences[:-10]

array(['Rly tragedy in MP: Some live to recount horror: \x89ÛÏWhen I saw coaches of my train plunging into water I called my daughters and said t...',
       'A river of lava in the sky this evening! It was indeed a beautiful sunset sky tonight. (8-4-15) http://t.co/17EGMlNi80',
       'Los Angeles Times: Arson suspect linked to 30 fires caught in Northern ... - http://t.co/xwMs1AWW8m #NewsInTweets http://t.co/TE2YeRugsi',
       ...,
       "When you lowkey already know you're gonna drown in school this year :) http://t.co/aCMrm833zq",
       '@JamesMelville Some old testimony of weapons used to promote conflicts\nTactics - corruption &amp; infiltration of groups\nhttps://t.co/cyU8zxw1oH',
       "Just got evacuated from the movie theatre for an emergency. Saw people running from another they're."],
      dtype=object)

In [14]:
train_labels[:-10]

array([1, 0, 1, ..., 0, 0, 1])

# Converting text data to numbers using tokenisation and embeddings (overview)
### Tokenization
- A straight mapping from word or character or sub-word to a numerical value. There are three main levels of tokenization:
1. Using word-level tokenization with the sentence "I love TensorFlow" might result in "I" being 0, "love" being 1 and "TensorFlow" being 2. In this case, every word in a sequence considered a single token.
2. Character-level tokenization, such as converting the letters A-Z to values 1-26. In this case, every character in a sequence considered a single token.
3. Sub-word tokenization is in between word-level and character-level tokenization. It involves breaking invidual words into smaller parts and then converting those smaller parts into numbers. For example, "my favourite food is pineapple pizza" might become "my, fav, avour, rite, fo, oo, od, is, pin, ine, app, le, piz, za". After doing this, these sub-words would then be mapped to a numerical value. In this case, every word could be considered multiple tokens.


### Embeddings
- An embedding is a representation of natural language which can be learned. Representation comes in the form of a feature vector. For example, the word "dance" could be represented by the 5-dimensional vector [-0.8547, 0.4559, -0.3332, 0.9877, 0.1112]. It's important to note here, the size of the feature vector is tuneable. There are two ways to use embeddings:
1. Create your own embedding - Once your text has been turned into numbers (required for an embedding), you can put them through an embedding layer (such as tf.keras.layers.Embedding) and an embedding representation will be learned during model training.
2. Reuse a pre-learned embedding - Many pre-trained embeddings exist online. These pre-trained embeddings have often been learned on large corpuses of text (such as all of Wikipedia) and thus have a good underlying representation of natural language. You can use a pre-trained embedding to initialize your model and fine-tune it to your own specific task.

# Setting up a TensorFlow TextVectorization layer to convert text to numbers

In [15]:
from tensorflow.keras.layers import TextVectorization

In [16]:
# these are the default parameters. this cell is just for demonstration
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (all of the different words in your text)
                                    standardize="lower_and_strip_punctuation", # how to process text
                                    split="whitespace", # how to split tokens
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None) # how long should the output sequence of tokens be?
                                    # pad_to_max_tokens=True) # Not valid if using max_tokens=None

In [17]:
# Find average number of tokens (words) in training Tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [18]:
# Setup text vectorization with custom variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

# Mapping the TextVectorization layer to text data and turning it into numbers

In [19]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

2023-05-15 23:10:17.411363: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [20]:
# Create sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[319,   3, 199,   4,  13, 699,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [21]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
The Murderous Story Of AmericaÛªs First Hijacking http://t.co/LK5uqKOP1e      

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   2, 2381,  366,    6, 3179,  100,  515,    1,    0,    0,    0,
           0,    0,    0,    0]])>

In [22]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # most common tokens (notice the [UNK] token for "unknown" words)
bottom_5_words = words_in_vocab[-5:] # least common tokens
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}")
print(f"Bottom 5 least common words: {bottom_5_words}")

# UNK is unknown token which replaces uncommon words. increasing max tokens will reduce it's popularity

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words: ['palmer', 'palm', 'palinfoen', 'palestinian\x89Û', 'paleface']


# Creating an Embedding layer to turn tokenised text into embedding vectors

In [23]:
embedding = tf.keras.layers.Embedding(input_dim=max_vocab_length,  # set input shape
                                      output_dim=128,  # a common starting point, multiples of 8 tend to run faster
                                      input_length=max_length,  # how long is the input?
                                      name="embedding_1")

In [24]:
# Get a random sentence from training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
REVEALED: Everton hijack United bid for 14-year-old WONDERKID! - http://t.co/nb1E7mNcE5      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.00855628, -0.03874923, -0.01302283, ...,  0.0338323 ,
         -0.01891991, -0.01729375],
        [-0.04195433, -0.00518826, -0.04065134, ...,  0.02791267,
         -0.00899041, -0.02700505],
        [-0.0340049 ,  0.02001712, -0.04924002, ...,  0.02281548,
          0.02247032,  0.02413313],
        ...,
        [-0.02395582, -0.04551792, -0.01203375, ...,  0.04350333,
         -0.00036246,  0.04597156],
        [-0.02395582, -0.04551792, -0.01203375, ...,  0.04350333,
         -0.00036246,  0.04597156],
        [-0.02395582, -0.04551792, -0.01203375, ...,  0.04350333,
         -0.00036246,  0.04597156]]], dtype=float32)>

In [25]:
# Check out a single token's embedding
display(sample_embed[0][0])
display(sample_embed[0][0].shape)
display(random_sentence)

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([-0.00855628, -0.03874923, -0.01302283, -0.03257175,  0.03298042,
       -0.02722744,  0.03294924, -0.01287933, -0.01725875, -0.00838133,
       -0.0464229 ,  0.03036087,  0.03990791, -0.04874191,  0.0169234 ,
       -0.00984835,  0.03581196, -0.01875039,  0.02582599, -0.04294046,
       -0.03127243,  0.00260308, -0.02231076,  0.04222877, -0.04466726,
       -0.0190603 ,  0.02879326, -0.03559661,  0.01852081, -0.04091055,
        0.04124311,  0.00541434, -0.01389253,  0.04096146,  0.03986054,
        0.01132878, -0.00135156, -0.03603177, -0.0401641 ,  0.02729185,
        0.03577474, -0.02339543,  0.03495786, -0.02730035, -0.0423415 ,
       -0.01723155, -0.027474  , -0.04930828, -0.04412589, -0.01237712,
       -0.02710778,  0.01847562,  0.03523394, -0.03215267,  0.03254438,
        0.04036567, -0.02455155,  0.02313561,  0.02951128, -0.0383168 ,
       -0.02115347,  0.00578755, -0.04267429, -0.0004696 , -0.03310784,
        0.017211

TensorShape([128])

'REVEALED: Everton hijack United bid for 14-year-old WONDERKID! - http://t.co/nb1E7mNcE5'

# the various modelling experiments we're going to run
- [Model 0: Sklearn Naive Bayes (baseline)](https://scikit-learn.org/stable/modules/naive_bayes.html)
- Model 1: Feed-forward neural network (dense model)
- Model 2: LSTM model
- Model 3: GRU model
- Model 4: Bidirectional-LSTM model
- Model 5: 1D Convolutional Neural Network
- Model 6: TensorFlow Hub Pretrained Feature Extractor
- Model 7: Same as model 6 with 10% of training data

# Model 0: Building a baseline model to try and improve upon [Sklearn Naive Bayes (baseline)](https://scikit-learn.org/stable/modules/naive_bayes.html)

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline (pipeline is a bit like tf.keras.models.Sequential()
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", MultinomialNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [27]:
# evaluate the model:
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 78.22%


In [28]:
# Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0])

# Creating a function to track and evaluate our model's results

In [29]:
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
    """
    Calculates model accuracy, precision, recall and f1 score of a binary classification model.
    only works for binary classification
    Args:
    -----
    y_true = true labels in the form of a 1D array
    y_pred = predicted labels in the form of a 1D array

    Returns a dictionary of accuracy, precision, recall, f1-score.
    """
    # Calculate model accuracy
    model_accuracy = accuracy_score(y_true, y_pred)
    # Calculate model precision, recall and f1 score using "weighted" average
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
    model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
    return model_results

In [30]:
# Baseline results
baseline_results = calculate_results(y_true=val_labels, y_pred=baseline_preds)
baseline_results

{'accuracy': 0.7821522309711286,
 'precision': 0.7922635210264124,
 'recall': 0.7821522309711286,
 'f1': 0.7730378142460546}

# Model 1: Building, fitting and evaluating our first deep model (feed forward) on text data

In [31]:
# Create tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create directory to save TensorBoard logs
SAVE_DIR = "model_logs"

In [32]:
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x)  # create embedding of numberized inputs
x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding
outputs = layers.Dense(1, activation="sigmoid")(x)

model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model

In [33]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [34]:
model_1.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [35]:
# Fit the model
model_1_history = model_1.fit(train_sentences, # input sentences can be a list of strings due to text preprocessing layer built-in model
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="simple_dense_model")])

Saving TensorBoard log files to: model_logs/simple_dense_model/20230515-231017
Epoch 1/5
215/215 [==============================] - 2s 9ms/step - loss: 0.6111 - accuracy: 0.6919 - val_loss: 0.5435 - val_accuracy: 0.7572
Epoch 2/5
215/215 [==============================] - 2s 11ms/step - loss: 0.4396 - accuracy: 0.8218 - val_loss: 0.4917 - val_accuracy: 0.7900
Epoch 3/5
215/215 [==============================] - 2s 7ms/step - loss: 0.3460 - accuracy: 0.8621 - val_loss: 0.4847 - val_accuracy: 0.7966
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.2829 - accuracy: 0.8899 - val_loss: 0.4971 - val_accuracy: 0.7953
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.2352 - accuracy: 0.9132 - val_loss: 0.5175 - val_accuracy: 0.7795


In [36]:
# Check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 814us/step - loss: 0.5175 - accuracy: 0.7795


[0.5174903869628906, 0.7795275449752808]

In [37]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[:10] # only print out the first 10 prediction probabilities

24/24 [==============================] - 0s 2ms/step


array([[0.9963592 ],
       [0.9372084 ],
       [0.8667326 ],
       [0.00447838],
       [0.28191525],
       [0.30955294],
       [0.85314935],
       [0.20639956],
       [0.12125789],
       [0.07120227]], dtype=float32)

In [38]:
# Turn prediction probabilities into single-dimension tensor of floats
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs)) # squeeze removes single dimensions
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0.,
       0., 1., 1.], dtype=float32)>

In [39]:
# Calculate model_1 metrics
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 0.7795275590551181,
 'precision': 0.7784534441870128,
 'recall': 0.7795275590551181,
 'f1': 0.7767394460307846}

In [40]:
# Is our simple Keras model better than our baseline model?
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))
# nope

array([False, False, False,  True])

# Visualizing our model's learned word embeddings with [TensorFlow's projector tool](https://projector.tensorflow.org/)

In [41]:
# redoing this from above for practice
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [42]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [43]:
# weight matrix of embedding layer
# (these are the numerical patterns between the text in the training dataset the model has learned)
embed_weights = model_1.get_layer("embedding_1").get_weights()[0]
print(embed_weights.shape) # same size as vocab size and embedding_dim (each word is a embedding_dim size vector)
print(3)

(10000, 128)
3


In [45]:
import io

# Create output writers
out_v = io.open("embedding_vectors.tsv", "w", encoding="utf-8")
out_m = io.open("embedding_metadata.tsv", "w", encoding="utf-8")

# Write embedding vectors and words to file
for num, word in enumerate(words_in_vocab):
  if num == 0:
     continue # skip padding token
  vec = embed_weights[num]
  out_m.write(word + "\n") # write words to file
  out_v.write("\t".join([str(x) for x in vec]) + "\n") # write corresponding word vector to file
out_v.close()
out_m.close()

# High-level overview of Recurrent Neural Networks (RNNs) + where to learn more
- RNNs are useful for sequence data
- The premise of an RNN is simple: use information from the past to help you with the future (this is where the term recurrent comes from). In other words, take an input (X) and compute an output (y) based on all previous inputs
### types of RNNs
- Long short-term memory cells (LSTMs).
- Gated recurrent units (GRUs).
- Bidirectional RNN's (passes forward and backward along a sequence, left to right and right to left).
### types of problems RNNs can be used in
- One to one: one input, one output, such as image classification.
- One to many: one input, many outputs, such as image captioning (image input, a sequence of text as caption output).
- Many to one: many inputs, one outputs, such as text classification (classifying a Tweet as real diaster or not real diaster).
- Many to many: many inputs, many outputs, such as machine translation (translating English to Spanish) or speech to text (audio wave as input, text as output)
### Resources
- [MIT Deep Learning Lecture on Recurrent Neural Networks](https://youtu.be/SEnXr6v2ifU)  - explains the background of recurrent neural networks and introduces LSTMs.
- [The Unreasonable Effectiveness of Recurrent Neural Networks by Andrej Karpathy](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) - demonstrates the power of RNN's with examples generating various sequences.
- [Understanding LSTMs by Chris Olah](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) - an in-depth (and technical) look at the mechanics of the LSTM cell, possibly the most popular RNN building block.

# Model 2: Building, fitting and evaluating our first TensorFlow RNN model (LSTM)

The reason we use a new embedding layer for each model is since the embedding layer is a learned representation of words (as numbers), if we were to use the same embedding layer (embedding_1) for each model, we'd be mixing what one model learned with the next. And because we want to compare our models later on, starting them with their own embedding layer each time is a better idea.

In [48]:
from tensorflow.keras import layers
model_2_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=128,
                                     embeddings_initializer="uniform",
                                     input_length=max_length,
                                     name="embedding_2")


# Create LSTM model
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = model_2_embedding(x)
print(x.shape)
# x = layers.LSTM(64, return_sequences=True)(x) # return vector for each word in the Tweet (you can stack RNN cells as long as return_sequences=True)
x = layers.LSTM(64)(x) # return vector for whole sequence
print(x.shape)
# x = layers.Dense(64, activation="relu")(x) # optional dense layer on top of output of LSTM cell
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

(None, 15, 128)
(None, 64)


In [49]:
model_2.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [50]:
model_2_history = model_2.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(train_sentences, train_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR, "model_2_experiment")])

Saving TensorBoard log files to: model_logs/model_2_experiment/20230515-233418
Epoch 1/5
215/215 [==============================] - 5s 19ms/step - loss: 0.5058 - accuracy: 0.7494 - val_loss: 0.2964 - val_accuracy: 0.8781
Epoch 2/5
215/215 [==============================] - 4s 16ms/step - loss: 0.3113 - accuracy: 0.8729 - val_loss: 0.1810 - val_accuracy: 0.9352
Epoch 3/5
215/215 [==============================] - 4s 17ms/step - loss: 0.2084 - accuracy: 0.9216 - val_loss: 0.1420 - val_accuracy: 0.9613
Epoch 4/5
215/215 [==============================] - 3s 16ms/step - loss: 0.1489 - accuracy: 0.9479 - val_loss: 0.1073 - val_accuracy: 0.9672
Epoch 5/5
215/215 [==============================] - 4s 18ms/step - loss: 0.1110 - accuracy: 0.9593 - val_loss: 0.0782 - val_accuracy: 0.9727


In [51]:
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  multiple                 0         
 ectorization)                                                   
                                                                 
 embedding_2 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [54]:
# Make predictions on the validation dataset
model_2_pred_probs = model_2.predict(val_sentences)
print(model_2_pred_probs.shape, model_2_pred_probs[:10]) # view the first 10

# Round out predictions and reduce to 1-dimensional array
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

24/24 [==============================] - 0s 2ms/step
(762, 1) [[0.9998538 ]
 [0.9812736 ]
 [0.999598  ]
 [0.00263736]
 [0.1972929 ]
 [0.02247726]
 [0.99982417]
 [0.09333376]
 [0.08962116]
 [0.00638218]]


<tf.Tensor: shape=(10,), dtype=float32, numpy=array([1., 1., 1., 0., 0., 0., 1., 0., 0., 0.], dtype=float32)>

In [56]:
# Calculate LSTM model results
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)
display(model_2_results)  # worse than baseline
display(baseline_results)

{'accuracy': 0.7335958005249343,
 'precision': 0.7355045899237286,
 'recall': 0.7335958005249343,
 'f1': 0.7343361743494092}

{'accuracy': 0.7821522309711286,
 'precision': 0.7922635210264124,
 'recall': 0.7821522309711286,
 'f1': 0.7730378142460546}